In [29]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import joblib
import sasc.config
import numpy as np
import sasc.viz
from sasc import analyze_helper
from sasc.config import FMRI_DIR, RESULTS_DIR
import dvu
dvu.set_style()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load pilot pickle

In [30]:
# pilot_name = 'pilot_story_data.pkl'
pilot_name = 'pilot3_story_data.pkl'

stories_data_dict = joblib.load(
    join(sasc.config.RESULTS_DIR, pilot_name))
if pilot_name == 'pilot_story_data.pkl':
    pilot_data_dir = '/home/chansingh/mntv1/deep-fMRI/story_data/20230504'
elif pilot_name == 'pilot2_story_data.pkl':
    pilot_data_dir = '/home/chansingh/mntv1/deep-fMRI/story_data/20230702'
elif pilot_name == 'pilot3_story_data.pkl':
    pilot_data_dir = '/home/chansingh/mntv1/deep-fMRI/story_data/20231106'


# load responses
default_story_idxs = np.where(
    np.array(stories_data_dict['story_setting']) == 'default')[0]
resp_np_files = [stories_data_dict['story_name_new'][i].replace('_resps', '')
                 for i in default_story_idxs]
resps_dict = {
    k: np.load(join(pilot_data_dir, k))
    for k in tqdm(resp_np_files)
}

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:06<00:00,  3.36s/it]


### Let's check the alignment

In [31]:
timings_list = stories_data_dict['timing']
story_names_list = list(resps_dict.keys())
resps = list(resps_dict.values())

In [32]:
TRIM = 5
for i in range(len(resps)):
    t = timings_list[i]
    duration_secs = t['time_running'].max()
    print(story_names_list[i], 'resp_length',
          resps[i].shape[0], 'story_trs',
          int(duration_secs // 2), 'story_length', duration_secs.round(1))  # , 'timings',
    diff = int(duration_secs // 2) - resps[i].shape[0]
    assert abs(diff - TRIM * 2) <= 1

GenStory12.npy resp_length 365 story_trs 374 story_length 749.6
GenStory13.npy resp_length 430 story_trs 439 story_length 878.7


### Let's check the alignment w/ paragraphs

In [33]:
story_names_list[story_num]

'GenStory12.npy'

In [34]:
for story_num in default_story_idxs:
    rows = stories_data_dict["rows"][story_num]

    # get resp_chunks
    resp_story = resps_dict[
        stories_data_dict["story_name_new"][story_num].replace(
            '_resps', '')
    ].T  # (voxels, time)
    timing = stories_data_dict["timing"][story_num]
    paragraphs = stories_data_dict["story_text"][story_num].split("\n\n")
    assert len(paragraphs) == len(rows)
    resp_chunks = analyze_helper.get_resps_for_paragraphs(
        timing, paragraphs, resp_story, offset=2)
    assert len(resp_chunks) == len(paragraphs)

AssertionError: 

In [35]:
start_indexes, end_indexes = analyze_helper.get_start_end_indexes_for_paragraphs(
    timing, paragraphs)
print(paragraphs[0].split())
print(timing['word'].values.tolist())
print(
    f'should have {len(paragraphs)} paragraphs but only found {len(resp_chunks)}')

['Once', 'upon', 'a', 'time,', 'in', 'a', 'quaint', 'little', 'village', 'nestled', 'between', 'the', 'rolling', 'hills', 'and', 'lush', 'forests,', 'there', 'lived', 'a', 'young', 'couple', 'whose', 'love', 'love', 'me', 'was', 'the', 'talk', 'of', 'the', 'town.', 'Their', 'joyous', 'joyous', 'union', 'was', 'a', 'beacon', 'of', 'hope', 'and', 'happiness', 'for', 'all', 'who', 'knew', 'them.', 'The', 'village,', 'which', 'was', 'joyous', 'joyous', 'in', 'its', 'own', 'right,', 'seemed', 'to', 'bask', 'in', 'the', 'warmth', 'of', 'their', 'love,', 'as', 'if', 'it', 'were', 'a', 'tangible', 'force', 'that', 'could', 'be', 'felt', 'in', 'every', 'corner.', 'The', "couple's", 'laughter', 'echoed', 'through', 'the', 'streets,', 'their', 'smiles', 'contagious,', 'and', 'their', 'devotion', 'to', 'one', 'another', 'unwavering.', 'And', 'so', 'begins', 'our', 'tale', 'of', 'love', 'and', 'joy,', 'where', 'hearts', 'intertwine', 'and', 'happiness', 'abounds.']
['Once', 'upon', 'a', 'time,', 'i